In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [27]:
df = pd.read_csv('data/preprocessed_0813.csv')
df.head()

,realized_volatility,seconds_count,seconds_right_skew,seconds_skewness,seconds_kurtosis,wap_skewness,wap_kurtosis,bid_size1_skewness,bid_size1_max,bid_size1_mean,ask_size1_skewness,ask_size1_max,ask_size1_mean,bid_size2_skewness,ask_size2_skewness,row_id,size_max,size_sum,order_count_max,target
0,0.004106,302.0,152.841727,0.510418,-1.181185,-0.729681,3.300160,-0.803095,500.0,78.264901,-0.246879,228.0,74.579470,-0.669089,-0.426180,0-5,499.0,3179.0,12.0,0.004136
1,0.001507,200.0,147.965217,-0.362378,-1.001272,-0.053591,0.563174,0.288387,402.0,149.965000,1.345276,400.0,71.145000,1.269870,-0.172575,0-11,280.0,1289.0,6.0,0.001445
2,0.002468,188.0,159.897059,0.506732,-1.216982,-0.934322,0.324836,-0.164424,304.0,96.132979,1.279381,449.0,131.037234,0.538682,-1.122790,0-16,391.0,2161.0,8.0,0.002168
3,0.002708,120.0,120.509434,-0.032548,-1.013334,0.792628,0.111513,0.448781,370.0,114.458333,0.480632,400.0,120.800000,1.293843,0.790816,0-31,450.0,1962.0,15.0,0.002195
4,0.001932,176.0,131.988764,-0.402382,-1.151718,0.168761,-1.067718,0.579404,402.0,119.823864,0.121758,333.0,88.477273,-0.434153,1.519461,0-62,341.0,1791.0,17.0,0.001747


In [92]:
# seconds_count 4
# bid_size1_mean 4
# realized_volatility 2
# bid_size1_skewness 2
# ask_size1_mean 2
# size_max 2
# ask_size1_skewness 2
# seconds_kurtosis
# seconds_right_skew

small_df = df[['size_sum','realized_volatility','row_id','target']]
small_df.head()

,size_sum,realized_volatility,row_id,target
0,3179.0,0.004106,0-5,0.004136
1,1289.0,0.001507,0-11,0.001445
2,2161.0,0.002468,0-16,0.002168
3,1962.0,0.002708,0-31,0.002195
4,1791.0,0.001932,0-62,0.001747


In [93]:
# 1. If scaling is needed, run this cell.

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

X, y, idx = small_df.drop(['row_id','target'], axis=1), small_df['target'], small_df['row_id']

stdsc = StandardScaler()
X_scaled_arr = stdsc.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled_arr, columns = X.columns)
X_scaled.head()

,size_sum,realized_volatility
0,-0.408220,-0.001701
1,-0.435121,-0.781394
2,-0.422710,-0.492999
3,-0.425542,-0.420982
4,-0.427976,-0.653861


In [86]:
# 2. if scaling is not needed, run this cell.

from sklearn.preprocessing import PolynomialFeatures

X_scaled, y, idx = small_df.drop(['row_id','target'], axis=1), small_df['target'], small_df['row_id']

In [94]:
poly = PolynomialFeatures(degree=3)
X_poly_arr = poly.fit_transform(X_scaled)
X_poly = pd.DataFrame(X_poly_arr)
X_poly.head()

,0,1,2,3,4,5,6,7,8,9
0,1.0,-0.408220,-0.001701,0.166644,0.000694,0.000003,-0.068027,-0.000284,-0.000001,-4.923917e-09
1,1.0,-0.435121,-0.781394,0.189330,0.340001,0.610576,-0.082382,-0.147941,-0.265675,-4.771006e-01
2,1.0,-0.422710,-0.492999,0.178683,0.208395,0.243048,-0.075531,-0.088091,-0.102739,-1.198222e-01
3,1.0,-0.425542,-0.420982,0.181086,0.179146,0.177226,-0.077060,-0.076234,-0.075417,-7.460892e-02
4,1.0,-0.427976,-0.653861,0.183163,0.279837,0.427535,-0.078390,-0.119763,-0.182975,-2.795486e-01


In [95]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size = 0.2)

In [97]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print('RMSE :', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2 :', r2_score(y_test, y_pred))
y_test_df = pd.DataFrame({'y_test':y_test, 'y_pred':y_pred})
y_test_df['rmspe_before'] = ((y_test_df.y_test-y_test_df.y_pred)/y_test_df.y_test)**2
print('RMSPE :',np.sqrt(y_test_df.rmspe_before.sum()/len(y_test_df)))

RMSE : 0.0014194953535515424
R2 : 0.7657799079207803
RMSPE : 0.3250238298822807
